In [1]:
import yaml
import os

default_entry = {
    "dbs":  "/sample/not/published",
    "era":  2018,
    "generator_weight":  1.0,
    "nevents":  -1,
    "nfiles":  -1,
    "nick":  "default",
    "sample_type":  "embedding",
    "xsec":  1.0,
}
finalstate_names = {
    "MuTau":"TauEmbedding-MuTauFinalState",
  "ElTau":"TauEmbedding-ElTauFinalState",
  "ElMu":"TauEmbedding-ElMuFinalState",
  "TauTau":"TauEmbedding-TauTauFinalState",
  "MuEmb": "MuonEmbedding",
  "ElEmb": "ElectronEmbedding",
}

default_database = "/work/sbrommer/smhtt_ul/ntuple_production/KingMaker_v4/sample_database/datasets.yaml"
with open(default_database, "r") as f:
    database = yaml.safe_load(f)
for filelist in os.listdir("ul_embedding_nano"):
    if filelist.endswith(".dbs"):
        run = filelist.split("_")[0]
        finalstate = filelist.split("_")[1].replace(".dbs", "")
        files = []
        events = 0
        prefix = ''
        with open(f"ul_embedding_nano/{filelist}", "r") as f:
            lines = f.readlines()
            preinfo = False
            for line in lines:
                if "events =" in line:
                    events = line.split("=")[1].strip()
                if "prefix =" in line:
                    prefix = line.split("=")[1].strip()
                    prefix = prefix.replace("/store/", "root://xrootd-cms.infn.it///store/")
                    preinfo = True
                    continue
                if preinfo:
                    files.append(prefix + '/' + line.split(" = ")[0].strip())
        nfiles = len(files)
        print(run, finalstate, nfiles, events)
        entry = default_entry.copy()
        era = int(run.replace("Run", "")[:4])
        entry["era"] = era
        entry["nick"] = f"{finalstate_names[finalstate]}_{run}-UL{era}"
        entry["nfiles"] = nfiles
        entry["nevents"] = int(events)
        entry["xsec"] = 1.0
        details = entry.copy()
        details["filelist"] = files
        if  not os.path.exists(f"ul_embedding_nano/test"):
            os.makedirs(f"ul_embedding_nano/test")
        with open(f"ul_embedding_nano/test/{run}_{finalstate}.yaml", "w") as f:
            yaml.dump(details, f)
        database[entry["nick"]] = (entry)
        # copy the filelist to the database
        basepath = "/work/sbrommer/ntuple_prototype/KingMaker/sample_database/"
        if not os.path.exists(f"{basepath}/{era}/embedding"):
            os.makedirs(f"{basepath}/{era}/embedding")
        os.system(f"cp ul_embedding_nano/test/{run}_{finalstate}.yaml {basepath}/{era}/embedding/{finalstate_names[finalstate]}_{run}-UL{era}.yaml")
with open(default_database, "w") as f:
        yaml.dump(database, f)


FileNotFoundError: [Errno 2] No such file or directory: '/work/sbrommer/ntuple_prototype/KingMaker/sample_database/datasets.yaml'